In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_target = pd.read_csv('data/df_targets.csv')

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_target['PRIM_CONTRIBUTORY_CAUSE'].values,
    test_size=0.2,
    random_state=42
)

# Convert the sparse matrices to PyTorch tensors
x_train = torch.tensor(x_train.toarray()).float()
x_test = torch.tensor(x_test.toarray()).float()
y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

# Define the logistic regression model
input_size = x_train.shape[1]
num_classes = len(pd.unique(df_target['PRIM_CONTRIBUTORY_CAUSE']))
model = nn.Linear(input_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


MemoryError: Unable to allocate 13.2 TiB for an array with shape (1193435, 1518686) and data type float64

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_target = pd.read_csv('data/df_targets.csv')

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_target['PRIM_CONTRIBUTORY_CAUSE'].values,
    test_size=0.2,
    random_state=42
)

# Convert the sparse matrices to PyTorch tensors
x_train = torch.tensor(x_train.toarray()).float()
x_test = torch.tensor(x_test.toarray()).float()
y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

# Define the logistic regression model
input_size = x_train.shape[1]
num_classes = len(pd.unique(df_target['PRIM_CONTRIBUTORY_CAUSE']))
model = nn.Linear(input_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


MemoryError: Unable to allocate 13.2 TiB for an array with shape (1193435, 1518686) and data type float64

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert the sparse matrix to PyTorch sparse tensor
x_train_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_train.indices), 
                                         torch.from_numpy(x_train.data),
                                         x_train.shape).float()
x_test_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_test.indices), 
                                        torch.from_numpy(x_test.data),
                                        x_test.shape).float()

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Create a custom dataset
train_dataset = TensorDataset(x_train_sparse, y_train_tensor)
test_dataset = TensorDataset(x_test_sparse, y_test_tensor)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the logistic regression model
input_dim = x_train.shape[1]
output_dim = len(set(y_train))
model = nn.Linear(input_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print("Test Accuracy:", accuracy)


RuntimeError: indices must be sparse_dim x nnz, but got: [56091445]

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Reshape the indices array
x_train_indices = x_train.indices.reshape(2, -1)
x_test_indices = x_test.indices.reshape(2, -1)

# Convert the sparse matrix to PyTorch sparse tensor
x_train_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_train_indices),
                                         torch.from_numpy(x_train.data),
                                         x_train.shape).float()
x_test_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_test_indices),
                                        torch.from_numpy(x_test.data),
                                        x_test.shape).float()

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Create a custom dataset
train_dataset = TensorDataset(x_train_sparse, y_train_tensor)
test_dataset = TensorDataset(x_test_sparse, y_test_tensor)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the logistic regression model
input_dim = x_train.shape[1]
output_dim = len(set(y_train))
model = nn.Linear(input_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print("Test Accuracy:", accuracy)


ValueError: cannot reshape array of size 56091445 into shape (2,newaxis)

In [6]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_target = pd.read_csv('data/df_targets.csv')

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_target['PRIM_CONTRIBUTORY_CAUSE'].values,
    test_size=0.2,
    random_state=42
)

# Convert the sparse matrix to PyTorch sparse tensor
x_train_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_train.indices),
                                         torch.from_numpy(x_train.data),
                                         x_train.shape).float()
x_test_sparse = torch.sparse_coo_tensor(torch.from_numpy(x_test.indices),
                                        torch.from_numpy(x_test.data),
                                        x_test.shape).float()

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train_sparse, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(df_target['PRIM_CONTRIBUTORY_CAUSE'].unique()))

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and data to the device
model = model.to(device)
x_train_sparse = x_train_sparse.to(device)
y_train_tensor = y_train_tensor.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    x_test_sparse = x_test_sparse.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # Forward pass
    logits = model(x_test_sparse)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


RuntimeError: indices must be sparse_dim x nnz, but got: [56091445]

In [10]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'])



# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_target['PRIM_CONTRIBUTORY_CAUSE'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(df_target['PRIM_CONTRIBUTORY_CAUSE'].unique()))

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and data to the device
model = model.to(device)
x_train = x_train.to(device)
y_train_tensor = y_train_tensor.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    x_test = x_test.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


ValueError: too many dimensions 'str'

In [2]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)


# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long().to(device)
y_test_tensor = torch.tensor(y_test).long().to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and data to the device
model = model.to(device)
x_train = x_train.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    x_test = x_test.to(device)

    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


KeyError: 'PRIM_CONTRIBUTORY_CAUSE_LABEL'

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Encode the target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long().to(device)
y_test_tensor = torch.tensor(y_test).long().to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and data to the device
model = model.to(device)
x_train = x_train.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    x_test = x_test.to(device)

    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


NameError: name 'device' is not defined

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Encode the target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model and data to the device
model = model.to(device)
x_train = x_train.to(device)
y_train_tensor = y_train_tensor.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    x_test = x_test.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


TypeError: 'int' object is not callable

In [2]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Encode the target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the data to the device
x_train = torch.tensor(x_train.toarray()).float().to(device)
x_test = torch.tensor(x_test.toarray()).float().to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


ValueError: y contains previously unseen labels: [25]

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the data to the device
x_train = torch.tensor(x_train.toarray()).float().to(device)
x_test = torch.tensor(x_test.toarray()).float().to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


MemoryError: Unable to allocate 92.6 GiB for an array with shape (59672, 208297) and data type float64

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the data to the device
x_train = torch.tensor(x_train.data.to(device)).float()
x_test = torch.tensor(x_test.data.to(device)).float()
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train


SyntaxError: invalid syntax (3818038347.py, line 67)

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the data to the device
x_train = torch.tensor(x_train.data.to(device)).float()
x_test = torch.tensor(x_test.data.to(device)).float()
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create sparse tensors and move to device
x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).to_dense().float()
x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).to_dense().float()
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


C:\Users\gerba\AppData\Local\Temp\ipykernel_34536\1089722546.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).to_dense().float()


RuntimeError: [enforce fail at C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 99435988672 bytes.

In [2]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the data to the device
x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).float()
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Create an instance of the logistic regression model
model = LogisticRegression(input_size=combined_sparse.shape[1], num_classes=len(label_encoder.classes_))

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)
